In [14]:
from scipy.io import loadmat

In [3]:
import numpy as np

In [4]:
import torch
from torchvision import models
import torch.nn as nn
from torchvision import transforms, datasets
import os
import torch.nn.functional as F
import torch.optim as optim
import time

In [5]:
def conv_dict(dict2):
    i = 0
    dict = {}
    for i in range(len(dict2)):
        if str(i) in dict2:
            if dict2[str(i)].shape == (0, 0):
                dict[str(i)] = dict2[str(i)]
            else:
                weights = dict2[str(i)][0]
                weights2 = []
                for weight in weights:
                    if weight.shape in [(1, x) for x in range(0, 5000)]:
                        weights2.append(weight[0])
                    else:
                        weights2.append(weight)
                dict[str(i)] = weights2
    return dict

In [6]:
def load_weights(weight_path):  # Function to load the model weights
    dict2 = loadmat(weight_path)
    dict1 = conv_dict(dict2)
    return dict1

In [7]:
def get_weight(weights, layer):
    weight = np.array(weights[layer])
    shape = weight[0].shape
    weight = np.reshape(weight[0], (shape[1], shape[0]))
    weight = torch.tensor(weight)
    return weight

In [8]:
weights = load_weights('./weights_L1L2.mat')

In [9]:
get_weight(weights, '4').shape

torch.Size([1, 32])

In [10]:
layer1 = nn.Linear(4096, 512)
layer1.weight.data = get_weight(weights, '0')

layer2 = nn.Linear(512, 32)
layer2.weight.data = get_weight(weights, '2')

layer3 = nn.Linear(32, 1)
layer3.weight.data = get_weight(weights, '4')

In [11]:
class anomaly_ann(nn.Module):
    def __init__(self):
        super(anomaly_ann, self).__init__()
        self.layer1 = nn.Linear(4096, 512)
        self.layer1.weight.data = get_weight(weights, '0')

        self.layer2 = nn.Linear(512, 32)
        self.layer2.weight.data = get_weight(weights, '2')

        self.layer3 = nn.Linear(32, 1)
        self.layer3.weight.data = get_weight(weights, '4')
        
    def forward(self, x):
        #print("x", x.shape)
        out = self.layer1(x)
        out = nn.ReLU(out)
        #print("after vgg", out.shape)
        out = self.layer2(out)
        #print("after transpose", out.shape)
        out = self.layer3(out)
        #print("after output layer", out.shape)
        return nn.Sigmoid(out)

In [12]:
net = anomaly_ann()

In [13]:
net

anomaly_ann(
  (layer1): Linear(in_features=4096, out_features=512, bias=True)
  (layer2): Linear(in_features=512, out_features=32, bias=True)
  (layer3): Linear(in_features=32, out_features=1, bias=True)
)